In [5]:
import pandas as pd
#import geopandas as gpd
import requests
import rasterio as rio
from rasterio.plot import show
from rasterio.warp import transform
from rasterio.transform import (xy, rowcol)
from rasterio.windows import Window
import itertools as it
import rioxarray as rx
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import time
import timeit
import os.path
from string import Template

from typing import Tuple, Optional, List

from leaf.deforestation import (
    cache_earthenginepartners_hansen,
    earthenginepartners_hansen)


In [14]:
df = pd.read_csv('data/assets_for_deforestation.csv', sep='\t')
df.country.value_counts().head(10)

# 'CN': ('China', (73.6753792663, 18.197700914, 135.026311477, 53.4588044297)),
# 'US': ('United States', (-171.791110603, 18.91619, -66.96466, 71.3577635769)),
# 'BR': ('Brazil', (-73.9872354804, -33.7683777809, -34.7299934555, 5.24448639569)),



country
China             10314
United States      1711
Germany            1673
India              1325
France             1007
Spain               865
United Kingdom      572
Japan               556
Brazil              527
Türkiye             343
Name: count, dtype: int64

In [ ]:
df[df.treecover2000 > 0]

In [ ]:
df[df['2014'] != 0].head()

In [16]:
# spanning the range 180W-180E and 80N-60S
# 80N, 70N, ... 10N, 00N, 10S, 20S, ... 50S - 180W
# 80N - 180W, 170W, ... 010W, 000E, 010E.... 170E

# https://worldpopulationreview.com/country-rankings/deforestation-rates-by-country
# https://gist.github.com/graydon/11198540
# 'BR': ('Brazil', (-73.9872354804, -33.7683777809, -34.7299934555, 5.24448639569)),
# 'ID': ('Indonesia', (95.2930261576, -10.3599874813, 141.03385176, 5.47982086834)),
# 'CD': ('Congo (Kinshasa)', (12.1823368669, -13.2572266578, 31.1741492042, 5.25608775474)),
# 'CG': ('Congo (Brazzaville)', (11.0937728207, -5.03798674888, 18.4530652198, 3.72819651938)),
# 'AO': ('Angola', (11.6400960629, -17.9306364885, 24.0799052263, -4.43802336998)),
# Tazmania
# 'MM': ('Myanmar', (92.3032344909, 9.93295990645, 101.180005324, 28.335945136)),
# 'PY': ('Paraguay', (-62.6850571357, -27.5484990374, -54.2929595608, -19.3427466773)),
# 'BO': ('Bolivia', (-69.5904237535, -22.8729187965, -57.4983711412, -9.76198780685)),
# 'MZ': ('Mozambique', (30.1794812355, -26.7421916643, 40.7754752948, -10.3170960425)),
# 'AR': ('Argentina', (-73.4154357571, -55.25, -53.628348965, -21.8323104794)),

# According to how many assets we have...

# long lat bottom left, long lat top right

# 'BR': ('Brazil', (-73.9872354804, -33.7683777809, -34.7299934555, 5.24448639569)),
earthenginepartners_hansen('data/assets_for_deforestation.csv', '\t', 
                           range(-33, 6, 10), range(-73, -35, -10), 
                           'data/assets_with_deforestation.csv', verbose=False)

# 'CN': ('China', (73.6753792663, 18.197700914, 135.026311477, 53.4588044297)),
earthenginepartners_hansen('data/assets_with_deforestation.csv', '\t', 
                           range(18, 54, 10), range(73, 136, 10), 
                           'data/assets_with_deforestation.csv', verbose=False)

# 'US': ('United States', (-171.791110603, 18.91619, -66.96466, 71.3577635769)),
earthenginepartners_hansen('data/assets_with_deforestation.csv', '\t', 
                           range(18, 72, 10), range(-171, -67, 10), 
                           'data/assets_with_deforestation.csv', verbose=False)



cache_earthenginepartners_hansen...
cache_earthenginepartners_hansen...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/robertnorris/Projects/eudr-exposure/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/z7/3j8p82y52y74r52zb1b0w8hr0000gn/T/ipykernel_56577/3732533419.py", line 29, in <module>
    earthenginepartners_hansen('data/assets_with_deforestation.csv', '\t',
  File "/Users/robertnorris/Projects/eudr-exposure/leaf/deforestation.py", line 564, in earthenginepartners_hansen
    df.to_csv(temp, sep=seperator)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/robertnorris/Projects/eudr-exposure/leaf/deforestation.py", line 321, in to_assets_with_treecover2000
    result = lookup(row=np_row, col=np_col, xdarray=xda)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/robertnorris/Projects/eudr-exposure/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py", line 2329, in __call__
    return self._vectorize_